In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from itertools import cycle
from pylab import *

from scipy import optimize
from sklearn.metrics import mean_squared_error,r2_score
%matplotlib inline
plt.rcParams['font.sans-serif']=['SimHei'] #Used to display Chinese labels normally
plt.rcParams['axes.unicode_minus']=False #Used to display the negative sign normally
import warnings
warnings.filterwarnings('ignore')
#import altair as alt

# MRIO computation

In [2]:
plt.rcParams['axes.linewidth'] = 1.5  #Figure width
# plt.rcParams['figure.dpi'] = 80  #Figure resolution
font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal'
        }
#'size': 10
plt.rc('font', **font)

In [3]:
#consumption input-output table
linkage_matrix=pd.read_excel('C:\\CityU\\wangjie\\原始表格(存在操作改动)\\居民消费碳排放系数计算2015-0813residence修改.xlsx',sheet_name='居民消费与行业部门连接')#可以通过sheet_name来指定读取的表单

In [4]:
# identifying the material
linkage_matrix_1 = linkage_matrix.iloc[3:,3:11]
linkage_matrix_1 = linkage_matrix_1.astype(float)
linkage_matrix_1 = linkage_matrix_1.reset_index(drop=True)

In [5]:
data_range = pd.read_excel('C:\\CityU\\wangjie\\原始表格(存在操作改动)\\0102重点部门供需端range.xlsx')#可以通过sheet_name来指定读取的表单

In [6]:
data_range1=data_range

In [8]:
MRIO2015_deflator=pd.read_csv('C:\\CityU\\wangjie\\原始表格(存在操作改动)\\新的表格\\MRIO2015_deflator.csv',encoding = "gbk",low_memory=False)#可以通过sheet_name来指定读取的表单

In [9]:
MRIO2015_deflator_1 = MRIO2015_deflator.iloc[2:1305,4:]

In [ ]:
#Output the diagonal matrix of carbon emission coefficients for each industry
#original_MRIO_deflator
#data dimension
#carbon_intensity
##residents_input_output:居民消费 
def industry_consumption_matrix(original_MRIO_deflator,n,carbon_intensity_2,residents_input_output):
    MRIO_deflator = original_MRIO_deflator.astype(float)
    #MRIO_deflator.reset_index(inplace=True)
 
    MRIO_deflator = MRIO_deflator.div(MRIO_deflator.iloc[-1][0:])
    MRIO_deflator.replace([np.inf, -np.inf, np.nan], 0, inplace=True)
    MRIO_deflator = MRIO_deflator[:-1]
    MRIO_deflator_out = np.array(MRIO_deflator)
#列昂惕夫逆矩阵
    t_c_matrix =np.eye(n, dtype=int)
    c_matrix=t_c_matrix-MRIO_deflator
    c_matrix_I = np.linalg.inv(c_matrix)#矩阵求逆
#Diagonal matrix of carbon emission coefficients of various industries
    coef_deff_indus = np.dot(carbon_intensity_2,c_matrix_I)#矩阵相乘
    diagonal_matrix = np.diagflat(coef_deff_indus)#输出对角线矩阵
#各行业碳排放矩阵   
    industry_consumption_matrix = np.dot(diagonal_matrix,residents_input_output)#矩阵相乘
    industry_consumption_matrix = pd.DataFrame(industry_consumption_matrix)

    return industry_consumption_matrix

# Simulation analysis

In [10]:
# Mean variance of different clusters
df1 = pd.read_excel('C:\\CityU\\wangjie\\new version\\20220414cluster占比&P595均值方差&消费情景-标准差修改.xlsx',sheet_name = 'low consumption-S6使用')

In [11]:
# Proportion of different clusters
df2 = pd.read_excel('C:\\CityU\\wangjie\\new version\\20220414cluster占比&P595均值方差&消费情景-标准差修改.xlsx',sheet_name = 'cluster占比修改')

In [12]:
#Scenario selection - produce the number of different clusters in different scenarios
def scenario(scenario,numbers):
    blank = []
    a = df2.iloc[0:160,[scenario]].values.reshape(-1).tolist()
    for i,k in zip(range(1,df1.shape[1],2),range(0,160)):
        #print (i)
        n = (a[k]/100)*numbers
        test = df1.iloc[2:12,[i,i+1]].values
        for j in range(0,test.shape[0]):
            #print (j)
            mu, sigma,num = test[j][0],test[j][1],round(n)
            s = np.random.normal(mu, sigma, num)
            blank.append(s)
    return blank

In [ ]:
a = scenario(6,100002)#情景and所有家户的cases总量是100000

In [ ]:
dta = pd.DataFrame()
for k,x in zip(range(0,1590,10),df2.分类):
    n = pd.DataFrame()
    for i in range(k,k+10):
        m =  pd.DataFrame(a[i])
        m1 = m.T
        n = n.append(m1)
    m2 = n.T
    m2.columns=['pfood','pdress','preside','pfacily','ptrco','phealth','prec','pother','fuel-share','transportation-share']
    m2['类型'] = x
    m2.loc[m2['pfood'] <=  0,'pfood']=0
    m2.loc[m2['pdress'] <=  0,'pdress']=0
    m2.loc[m2['preside'] <=  0,'preside']=0
    m2.loc[m2['pfacily'] <=  0,'pfacily']=0
    m2.loc[m2['ptrco'] <=  0,'ptrco']=0
    m2.loc[m2['phealth'] <=  0,'phealth']=0
    m2.loc[m2['prec'] <=  0,'prec']=0
    m2.loc[m2['pother'] <=  0,'pother']=0
    m2.loc[m2['pother'] <=  0,'pother']=0
    m2.loc[m2['fuel-share'] <=  0,'fuel-share']=0
    m2.loc[m2['transportation-share'] <=  0,'transportation-share']=0
    dta = dta.append(m2)

In [ ]:
#dta.to_excel('C:\\CityU\\wangjie\\new version\\4-15模拟结果\\S6.xlsx')